(1)
https://github.com/MiaHanzhangYuan/Lab3.git

(2)

In [2]:
import pandas as pd
import numpy as np
import requests
import os
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [31]:
url ='https://databank.worldbank.org/data/download/ESG_CSV.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [5]:
url = 'https://v-dem.net/media/datasets/V-Dem-CY-Core_csv_v13.zip'
r = requests.get(url)
print(r)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


<Response [200]>


In [125]:
wb = pd.read_csv("ESGData.csv")
country = pd.read_csv("ESGCountry.csv")

(c)

In [126]:
vdem = pd.read_csv("V-Dem-CY-Core-v13.csv")

In [127]:
vdem = vdem[['country_text_id', 'country_name', 'year', 'v2x_polyarchy']]

In [128]:
vdem.head(3).T

,0,1,2
country_text_id,MEX,MEX,MEX
country_name,Mexico,Mexico,Mexico
year,1789,1790,1791
v2x_polyarchy,0.028,0.028,0.028


In [129]:
vdem = vdem.query("year>=1960 & year<=2021")

In [130]:
vdem = vdem.rename({'country_text_id':'country_code',
                   'country_name':'country_name_vdem',
                   'v2x_polyarchy':'democracy'}, axis=1)

In [131]:
vdem = vdem.sort_values(by = ['country_code', 'year'], ascending = True)

(d)

In [79]:
country = country[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', 'Income Group']]

In [80]:
country.columns = [x.lower() for x in country.columns]
country.columns = [x.replace(' ','_') for x in country.columns]
country.columns

Index(['country_code', 'table_name', 'long_name', 'currency_unit', 'region',
       'income_group'],
      dtype='object')

In [81]:
country = country.rename({'table_name':'country_name_wb'}, axis=1)
country

,country_code,country_name_wb,long_name,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


In [82]:
noncountries = ["Arab World", "Central Europe and the Baltics",
 "Caribbean small states",
 "East Asia & Pacific (excluding high income)",
 "Early-demographic dividend","East Asia & Pacific",
 "Europe & Central Asia (excluding high income)",
 "Europe & Central Asia", "Euro area",
 "European Union","Fragile and conflict affected situations",
 "High income",
 "Heavily indebted poor countries (HIPC)","IBRD only",
 "IDA & IBRD total",
 "IDA total","IDA blend","IDA only",
 "Latin America & Caribbean (excluding high income)",
 "Latin America & Caribbean",
 "Least developed countries: UN classification",
 "Low income","Lower middle income","Low & middle income",
 "Late-demographic dividend","Middle East & North Africa",
 "Middle income",
 "Middle East & North Africa (excluding high income)",
 "North America","OECD members",
 "Other small states","Pre-demographic dividend",
 "Pacific island small states",
 "Post-demographic dividend",
 "Sub-Saharan Africa (excluding high income)","Sub-Saharan Africa",
 "Small states","East Asia & Pacific (IDA & IBRD)",
 "Europe & Central Asia (IDA & IBRD)",
 "Latin America & Caribbean (IDA & IBRD)",
 "Middle East & North Africa (IDA & IBRD)","South Asia",
 "South Asia (IDA & IBRD)",
 "Sub-Saharan Africa (IDA & IBRD)",
 "Upper middle income", "World"]

In [83]:
country.query("country_name_wb not in @noncountries")

,country_code,country_name_wb,long_name,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


(e)

In [134]:
filter_col = [col for col in wb if col.startswith('19') or col.startswith('20')]

In [135]:
wb = wb[['Country Code', 'Country Name', 'Indicator Code'] + filter_col]

In [136]:
wb.head(3)

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab World,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab World,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab World,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN


In [137]:
noncountries.remove('World')

ValueError: list.remove(x): x not in list

In [138]:
replace_map = {
 "AG.LND.AGRI.ZS": "agricultural_land",
 "AG.LND.FRST.ZS": "forest_area",
 "AG.PRD.FOOD.XD": "food_production_index",
 "CC.EST": "control_of_corruption",
 "EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
 "EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
 "EG.ELC.ACCS.ZS": "access_to_electricity",
 "EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
 "EG.ELC.RNEW.ZS": "renewable_electricity_output",
 "EG.FEC.RNEW.ZS": "renewable_energy_consumption",
 "EG.IMP.CONS.ZS": "energy_imports",
 "EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
 "EG.USE.PCAP.KG.OE": "energy_use",
 "EN.ATM.CO2E.PC": "co2_emissions",
 "EN.ATM.METH.PC": "methane_emissions",
 "EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
 "EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
 "EN.CLC.CDDY.XD": "cooling_degree_days",
 "EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
 "EN.CLC.HEAT.XD": "heat_index_35",
 "EN.CLC.MDAT.ZS": "droughts",
 "EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
 "EN.CLC.SPEI.XD": "mean_drought_index","EN.MAM.THRD.NO": "mammal_species",
 "EN.POP.DNST": "population_density",
 "ER.H2O.FWTL.ZS": "annual_freshwater_withdrawals",
 "ER.PTD.TOTL.ZS": "terrestrial_and_marine_protected_areas",
 "GB.XPD.RSDV.GD.ZS": "research_and_development_expenditure",
 "GE.EST": "government_effectiveness",
 "IC.BUS.EASE.XQ": "ease_of_doing_business_rank",
 "IC.LGL.CRED.XQ": "strength_of_legal_rights_index",
 "IP.JRN.ARTC.SC": "scientific_and_technical_journal_articles",
 "IP.PAT.RESD": "patent_applications",
 "IT.NET.USER.ZS": "individuals_using_the_internet",
 "NV.AGR.TOTL.ZS": "agriculture",
 "NY.ADJ.DFOR.GN.ZS": "net_forest_depletion",
 "NY.ADJ.DRES.GN.ZS": "natural_resources_depletion",
 "NY.GDP.MKTP.KD.ZG": "gdp_growth",
 "PV.EST": "political_stability_and_absence_of_violence",
 "RL.EST": "rule_of_law",
 "RQ.EST": "regulatory_quality",
 "SE.ADT.LITR.ZS": "literacy_rate",
 "SE.ENR.PRSC.FM.ZS": "gross_school_enrollment",
 "SE.PRM.ENRR": "primary_school_enrollment",
 "SE.XPD.TOTL.GB.ZS": "government_expenditure_on_education",
 "SG.GEN.PARL.ZS": "proportion_of_seats_held_by_women_in_national_parliamen",
 "SH.DTH.COMM.ZS": "cause_of_death",
 "SH.DYN.MORT": "mortality_rate",
 "SH.H2O.SMDW.ZS": "people_using_safely_managed_drinking_water_services",
 "SH.MED.BEDS.ZS": "hospital_beds",
 "SH.STA.OWAD.ZS": "prevalence_of_overweight",
 "SH.STA.SMSS.ZS": "people_using_safely_managed_sanitation_services",
 "SI.DST.FRST.20": "income_share_held_by_lowest_20pct",
 "SI.POV.GINI": "gini_index",
 "SI.POV.NAHC": "poverty_headcount_ratio_at_national_poverty_lines",
 "SI.SPR.PCAP.ZG": "annualized_average_growth_rate_in_per_capita_real_surve",
 "SL.TLF.0714.ZS": "children_in_employment",
 "SL.TLF.ACTI.ZS": "labor_force_participation_rate",
 "SL.TLF.CACT.FM.ZS": "ratio_of_female_to_male_labor_force_participation_ra",
 "SL.UEM.TOTL.ZS": "unemployment",
 "SM.POP.NETM": "net_migration",
 "SN.ITK.DEFC.ZS": "prevalence_of_undernourishment",
 "SP.DYN.LE00.IN": "life_expectancy_at_birth",
 "SP.DYN.TFRT.IN": "fertility_rate",
 "SP.POP.65UP.TO.ZS": "population_ages_65_and_above",
 "SP.UWT.TFRT": "unmet_need_for_contraception",
 "VA.EST": "voice_and_accountability",
 "EN.CLC.CSTP.ZS": "coastal_protection",
 "SD.ESR.PERF.XQ": "economic_and_social_rights_performance_score",
 "EN.CLC.HDDY.XD": "heating_degree_days",
 "EN.LND.LTMP.DC": "land_surface_temperature",
 "ER.H2O.FWST.ZS": "freshwater_withdrawal",
 "EN.H2O.BDYS.ZS": "water_quality",
 "AG.LND.FRLS.HA": "tree_cover_loss",
}

In [139]:
wb = wb.replace(replace_map)

In [140]:
wb

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab World,access_to_clean_fuels_and_technologies_for_coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab World,access_to_electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab World,natural_resources_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN
3,ARB,Arab World,net_forest_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062081,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN
4,ARB,Arab World,agricultural_land,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,31.0504,...,39.838650,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,ZWE,Zimbabwe,tree_cover_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,ZWE,Zimbabwe,unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,ZWE,Zimbabwe,unmet_need_for_contraception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(f)

In [141]:
wb1 = pd.melt(wb, id_vars=wb.columns[:3],value_vars = wb.columns[3:])

In [142]:
wb1 = wb1.rename({'variable':'year', 'value':'features'}, axis=1)

In [143]:
wb1

,Country Code,Country Name,Indicator Code,year,features
0,ARB,Arab World,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN
1,ARB,Arab World,access_to_electricity,1960,NaN
2,ARB,Arab World,natural_resources_depletion,1960,NaN
3,ARB,Arab World,net_forest_depletion,1960,NaN
4,ARB,Arab World,agricultural_land,1960,NaN
...,...,...,...,...,...
1069042,ZWE,Zimbabwe,terrestrial_and_marine_protected_areas,2022,27.214747
1069043,ZWE,Zimbabwe,tree_cover_loss,2022,NaN
1069044,ZWE,Zimbabwe,unemployment,2022,7.945000
1069045,ZWE,Zimbabwe,unmet_need_for_contraception,2022,NaN


(g)

In [144]:
wb1["year"] = wb1["year"].astype(int)

(h)

In [145]:
wb = pd.read_csv("ESGData.csv")

In [146]:
wb

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN,NaN
2,Arab World,ARB,Adjusted savings: natural resources depletion ...,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN,NaN
3,Arab World,ARB,Adjusted savings: net forest depletion (% of GNI),NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN,NaN
4,Arab World,ARB,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,...,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,Zimbabwe,ZWE,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747,NaN
16965,Zimbabwe,ZWE,Tree Cover Loss (hectares),AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,Zimbabwe,ZWE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000,NaN
16967,Zimbabwe,ZWE,Unmet need for contraception (% of married wom...,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,...,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
wb2 = wb[wb['Country Name']=='World']

In [148]:
wb2 = wb2.drop(labels = ['Country Code', 'Country Name'],axis =1)

In [149]:
wb2 = wb2.rename({'Indicator Name':'World_Indicator Name', 'Indicator Code':'World_Indicator Code'}, axis=1)

(i)

In [155]:
wb1 = wb1.rename({'Country Code':'country_code'},axis = 1)

merge1 = pd.merge(wb1, vdem,
                on = ['country_code', 'year'],
                how = 'outer',
                validate = 'many_to_many',
                indicator = 'matched')

In [156]:
merge1

,country_code,Country Name,Indicator Code,year,features,country_name_vdem,democracy,matched
0,ARB,Arab World,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN,NaN,NaN,left_only
1,ARB,Arab World,access_to_electricity,1960,NaN,NaN,NaN,left_only
2,ARB,Arab World,natural_resources_depletion,1960,NaN,NaN,NaN,left_only
3,ARB,Arab World,net_forest_depletion,1960,NaN,NaN,NaN,left_only
4,ARB,Arab World,agricultural_land,1960,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
1069437,ZZB,NaN,NaN,2017,NaN,Zanzibar,0.267,right_only
1069438,ZZB,NaN,NaN,2018,NaN,Zanzibar,0.268,right_only
1069439,ZZB,NaN,NaN,2019,NaN,Zanzibar,0.266,right_only
1069440,ZZB,NaN,NaN,2020,NaN,Zanzibar,0.258,right_only


In [157]:
merge1['matched'].value_counts()

matched
both          708296
left_only     360751
right_only       395
Name: count, dtype: int64

(j)

In [161]:
vdemonly = merge1.query("matched =='right_only'")

In [165]:
vdemmax = vdemonly.groupby(['country_code', 'country_name_vdem'])\
.agg({'year':'max'}).reset_index()


In [166]:
vdemmin = vdemonly.groupby(['country_code', 'country_name_vdem'])\
.agg({'year':'min'}).reset_index()

In [ ]:
wbonly = merge1.query("matched =='left_only'")

In [167]:
wbmax = wbonly.groupby(['country_code', 'Country Name'])\
.agg({'year':'max'}).reset_index()


In [168]:
wbmin = wbonly.groupby(['country_code', 'Country Name'])\
.agg({'year':'min'}).reset_index()


In [171]:
vdemmax

,country_code,country_name_vdem,year
0,DDR,German Democratic Republic,1990
1,HKG,Hong Kong,2021
2,PSE,Palestine/West Bank,2021
3,PSG,Palestine/Gaza,2021
4,SML,Somaliland,2021
5,TWN,Taiwan,2021
6,VDR,Republic of Vietnam,1975
7,XKX,Kosovo,2021
8,YMD,South Yemen,1990
9,ZZB,Zanzibar,2021


In [170]:
wbmin

,country_code,Country Name,year
0,AFG,Afghanistan,2022
1,AGO,Angola,2022
2,ALB,Albania,2022
3,AND,Andorra,1960
4,ARB,Arab World,1960
...,...,...,...
234,WSM,Samoa,1960
235,YEM,"Yemen, Rep.",2022
236,ZAF,South Africa,2022
237,ZMB,Zambia,2022


(l)

In [ ]:
merge1 = pd.merge(wb1, vdem,
                on = ['country_code', 'year'],
                how = 'outer',
                validate = 'many_to_many')
